In [1]:
import hopsworks

In [2]:
HOPSWORKS_PROJECT_NAME = 'taxi_demand_hwfs'

In [3]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [4]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/241814


In [5]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [12]:

FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1
feature_group = feature_store.get_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION
)

In [18]:
# Custom code - To get the count of features from a fs group using get_statistics

fs = feature_group.get_statistics(commit_time=None)

fs.content


{'columns': [{'approximateNumDistinctValues': 15903,
   'isDataTypeInferred': 'false',
   'dataType': 'String',
   'column': 'pickup_hour',
   'numRecordsNull': 0,
   'completeness': 1,
   'numRecordsNonNull': 4235760},
  {'dataType': 'Integral',
   'column': 'rides',
   'numRecordsNull': 0,
   'sum': 70736824,
   'completeness': 1,
   'approximateNumDistinctValues': 649,
   'isDataTypeInferred': 'false',
   'mean': 16.699913120667837,
   'maximum': 1115,
   'stdDev': 50.46135035982242,
   'minimum': 0,
   'numRecordsNonNull': 4235760,
   'approxPercentiles': []},
  {'dataType': 'Integral',
   'column': 'pickup_location_id',
   'numRecordsNull': 0,
   'sum': 563356080.0,
   'completeness': 1,
   'approximateNumDistinctValues': 267,
   'isDataTypeInferred': 'false',
   'mean': 133,
   'maximum': 265,
   'stdDev': 76.49836599562113,
   'minimum': 1,
   'numRecordsNonNull': 4235760,
   'approxPercentiles': []},
  {'dataType': 'Integral',
   'column': 'pickup_ts',
   'numRecordsNull': 0,
 

In [17]:

# Find statistics for 'pickup_hour' column from the above fs.content
pickup_hour_stats = next((col for col in fs.content['columns'] if col['column'] == 'pickup_hour'), None)

# Print counts and sum if statistics found
if pickup_hour_stats:
    num_records_null = pickup_hour_stats['numRecordsNull']
    num_records_non_null = pickup_hour_stats['numRecordsNonNull']

    print(f"Number of records with null values for 'pickup_hour': {num_records_null}")
    print(f"Number of records with non-null values for 'pickup_hour': {num_records_non_null}")
    print(f"Sum of null and non-null records: {num_records_null + num_records_non_null}")
else:
    print("Statistics for 'pickup_hour' column not found in the content.")

Number of records with null values for 'pickup_hour': 0
Number of records with non-null values for 'pickup_hour': 4235760
Sum of null and non-null records: 4235760


In [19]:
# *** Cautious : Use delete() only if you really need it ***
# Delete Feature View
feature_view = feature_store.get_feature_view("time_series_hourly_feature_view",version=1)
#feature_view.delete()

# Order of feature group and feature view deletion: 1st feature view accosiated with the fg and then fg.
# Here Parent is fg and child is fv, first delete child then parent if needed; but 
# not vice versa to avoid compatibility issues.

In [20]:
feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Error: Reading data from Hopsworks, using Hive           


KeyboardInterrupt: 

In [10]:
# Specify the feature group name and version to delete
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

# Get the feature group object with the name and version
feature_group = feature_store.get_feature_group(FEATURE_GROUP_NAME,FEATURE_GROUP_VERSION)

In [11]:
# *** Cautious : Use delete() only if you really need it ***
# Delete the feature group 
feature_group.delete()

JobWarning: All jobs associated to feature group `time_series_hourly_feature_group`, version `2` will be removed.

ValueError: Cannot get back the feature view because the query defined is no longer valid. Some feature groups used in the query may have been deleted. You can clean up this feature view on the UI or `FeatureView.clean`.